# Use Spark Streaming, SQL, and ML with iguazio
Spark users can access files, tables or streams stored on iguazio data platform through the native spark Dataframe interfaces. <br>
iguazio drivers for Spark implement the data-source API and allow `predicate push down` (the queries pass to iguazio database which only return the relevant data), this allow accelerated and high-speed access from Spark to data stored in iguazio DB. for more details read [Spark API documentation]()

## loading a file from AWS S3 into iguazio file system  


In [ ]:
%%sh 
mkdir -p $HOME/examples
curl -L "deutsche-boerse-xetra-pds.s3.amazonaws.com/2018-03-26/2018-03-26_BINS_XETR07.csv" > $HOME/examples/stocks.csv


## Initiating a Spark session 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Working with Spark notebook").getOrCreate()

# Data Source
To use the Iguazio Spark connector to read or write NoSQL data in the platform, use the format method to set the DataFrame’s data-source format to the platform’s custom NoSQL data source — "io.iguaz.v3io.spark.sql.kv". See the following read and write examples:

## Read the csv file using Spark DF
You can use the custom NoSQL DataFrame inferSchema read option to automatically infer the schema of the read table from its contents.

In [2]:
df = spark.read.option("inferSchema", "true").option("header", "true").csv('v3io://users/iguazio/examples/stocks.csv')
df.show()

+------------+--------+--------------------+------------+--------+----------+-------------------+-----+----------+--------+--------+--------+------------+--------------+
|        ISIN|Mnemonic|        SecurityDesc|SecurityType|Currency|SecurityID|               Date| Time|StartPrice|MaxPrice|MinPrice|EndPrice|TradedVolume|NumberOfTrades|
+------------+--------+--------------------+------------+--------+----------+-------------------+-----+----------+--------+--------+--------+------------+--------------+
|AT0000A0E9W5|    SANT|S+T AG (Z.REG.MK....|Common stock|     EUR|   2504159|2018-03-26 00:00:00|07:00|     20.56|   20.56|   20.56|   20.56|        1115|             5|
|DE000A0WMPJ6|    AIXA|  AIXTRON SE NA O.N.|Common stock|     EUR|   2504428|2018-03-26 00:00:00|07:00|    17.035|   17.08|   16.92|   16.98|        2892|            11|
|DE000A0Z2XN6|     RIB|RIB SOFTWARE SE  ...|Common stock|     EUR|   2504436|2018-03-26 00:00:00|07:00|     24.02|   24.18|   23.94|   24.12|        5

## Writing the spark DF into a table in Iguazio DB
Specify the path to the NoSQL table that is associated with the DataFrame as a fully qualified path of the format v3io://container name/data path —
where container name is the name of the table’s parent container, and data path is the path to the data within the specified container

In [3]:
# specify the DB index key using the key option (note the key must be unique)
df.write.format("io.iguaz.v3io.spark.sql.kv").mode("append").option("key", "ISIN").save("v3io://users/iguazio/examples/stocks_tab")


## Reading a table via Spark DF

In [4]:
spark.read.format("io.iguaz.v3io.spark.sql.kv").load("v3io://users/iguazio/examples/stocks_tab").show()

+------------+--------+--------------------+------------+--------+----------+-------------------+-----+----------+--------+--------+--------+------------+--------------+
|        ISIN|Mnemonic|        SecurityDesc|SecurityType|Currency|SecurityID|               Date| Time|StartPrice|MaxPrice|MinPrice|EndPrice|TradedVolume|NumberOfTrades|
+------------+--------+--------------------+------------+--------+----------+-------------------+-----+----------+--------+--------+--------+------------+--------------+
|DE0005933964|    EXI1|ISHARES SLI UCITS...|         ETF|     EUR|   2504990|2018-03-26 00:00:00|07:05|     80.38|   80.38|   80.38|   80.38|         417|             1|
|IE00B1FZS350|    IQQ6|ISHSII-DEV.MKT.PR...|         ETF|     EUR|   2505587|2018-03-26 00:00:00|07:07|      19.7|    19.7|    19.7|    19.7|         500|             1|
|DE0005008007|     ADL|ADLER REAL ESTATE AG|Common stock|     EUR|   2504874|2018-03-26 00:00:00|07:02|     13.22|   13.22|   13.22|   13.22|         

## Writing to Parquet

In [5]:
df.write.parquet("v3io://users/iguazio/examples/stocks_tab.parquet")

## Table Schema-Overwrite Examples
The following example creates a table named mytable with AttrA and AttrB attributes of type string and an AttrC attribute of type long, and then overwrites the table schema to change the type of AttrC to double:

In [6]:
df = spark.createDataFrame([
    ("a", "z", 123),
    ("b", "y", 456)
], ["AttrA", "AttrB", "AttrC"])
df.write.format("io.iguaz.v3io.spark.sql.kv") \
    .mode("overwrite") \
    .option("key", "AttrA") \
    .save("v3io://users/iguazio/examples/mytable/")
    
df = spark.createDataFrame([
    ("c", "x", 32.12),
    ("d", "v", 45.2)
], ["AttrA", "AttrB", "AttrC"])
df.write.format("io.iguaz.v3io.spark.sql.kv") \
    .mode("append") \
    .option("key", "AttrA") \
    .option("allow-overwrite-schema", "true") \
    .save("v3io://users/iguazio/examples/mytable/")

## Creating a partition table

This examples creates a partitioned “weather” table  The option("partition", "year, month, day") write option partitions the table by the year, month, and day item attributes. As demonstrated in the following image, if you browse the container in the dashboard after running the example, you’ll see that the weather directory has year=<value>/month=<value>/day=<value> partition directories that match the written items. If you select any of the nested day partition directories, you can see the written items and their attributes. For example, the first item (with attribute values 2016, 3, 25, 6, 16, 0.00, 55) is saved to a 20163256 file in a weather/year=2016/month=3/day=25 partition directory.

In [ ]:
table_path = "v3io://users/iguazio/examples/weather/"

df = spark.createDataFrame([
    (2016,  3, 25, 17, 18, 0.2, 62),
    (2016,  7, 24,  7, 19, 0.0, 52),
    (2016, 12, 24,  9, 10, 0.1, 47),
    (2017,  5,  7, 14, 21, 0.0, 70),
    (2017, 11,  1, 10, 15, 0.0, 34),
    (2017, 12, 12, 16, 12, 0.0, 47),
    (2017, 12, 24, 17, 11, 1.0, 50),
    (2018,  1, 18, 17, 10, 2.0, 45),
    (2018,  5, 20, 21, 20, 0.0, 59),
    (2018, 11,  1, 11, 11, 0.1, 65)
], ["year", "month", "day", "hour", "degrees_cel", "rain_ml", "humidity_per"])
df_with_key = df.withColumn(
    "time", concat(df["year"], df["month"], df["day"], df["hour"]))
df_with_key.write.format("io.iguaz.v3io.spark.sql.kv") \
    .mode("overwrite") \
    .option("key", "time") \
    .option("partition", "year, month, day, hour") \
    .save(table_path)

## Reading from partition table
Following is the output of the example’s show commands for each read. The filtered results are gathered by scanning only the partition directories that match the filter criteria.


### Full table read

In [ ]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path)
readDF.show()

### month < 7 filter — retrieve all data for the first six months of each year:

In [ ]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path) \
    .filter("month > 6")
readDF.show()

### month == 12 AND day == 24 filter — retrieve all hours on Dec 24 each year:

In [ ]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path) \
    .filter("month == 12 AND day == 24")
readDF.show()

### month > 6 AND hour >= 8 AND hour <= 20 filter — retrieve 08:00–20:00 data for every day in the last six months of each year:

In [ ]:

readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path) \
    .filter("month < 7 AND hour >= 8 AND hour <= 20")
readDF.show()

## Conditional update
This example demonstrates how to conditionally update NoSQL table items by using the condition write option. Each write call in the example is followed by matching read and show calls to read and display the value of the updated item in the target table after the write operation.

The first write command writes an item (row) to a “cars” table . The item’s reg_license primary-key (identity-column) attribute is set to 7843321, the mode attribute is set to “Honda”, and the odometer attribute is set to 29321. The overwrite save mode is used to overwrite the table if it already exists and create it otherwise. Reading the item from the table produces this output:

In [11]:
writeDF = spark.createDataFrame([("7843321", "Honda", 29321)],
                                ["reg_license", "model", "odometer"])
writeDF.write.format("io.iguaz.v3io.spark.sql.kv") \
    .option("key", "reg_license") \
    .mode("overwrite").save("v3io://users/iguazio/examples/cars/")
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv") \
    .load("v3io://users/iguazio/examples/cars/")
readDF.show()

writeDF = spark.createDataFrame([("7843321", "Honda", 31718)],
                                ["reg_license", "model", "odometer"])
writeDF.write.format("io.iguaz.v3io.spark.sql.kv") \
    .option("key", "reg_license") \
    .option("condition", "${odometer} > odometer") \
    .mode("append").save("v3io://users/iguazio/examples/cars/")
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv") \
    .load("v3io://users/iguazio/examples/cars/")
readDF.show()



+-----------+-----+--------+
|reg_license|model|odometer|
+-----------+-----+--------+
|    7843321|Honda|   29321|
+-----------+-----+--------+

+-----------+-----+--------+
|reg_license|model|odometer|
+-----------+-----+--------+
|    7843321|Honda|   31718|
+-----------+-----+--------+



# Using SQL queries (using Presto)
## Reading the stock_tab table using SQL after being written by Spark DF


In [12]:
%sql select * from v3io.users."/iguazio/examples/stocks_tab" where tradedvolume > 20000

Done.


securitydesc,securitytype,time,isin,minprice,date,endprice,numberoftrades,mnemonic,currency,securityid,maxprice,tradedvolume,startprice
AMUNDI ETF MSCI EMER.MKTS,ETF,07:04,FR0010959676,4.1296,2018-03-26 00:00:00.000,4.1296,2,AMEM,EUR,2505311,4.1296,57117,4.1296
COMMERZBANK ETC UNL.,ETC,07:11,DE000ETC0308,0.284,2018-03-26 00:00:00.000,0.284,1,X0D2,EUR,2506314,0.284,30000,0.284
XTR.MSCI BANGL.SWAP 1CDL,ETF,07:05,LU0659579220,0.8917,2018-03-26 00:00:00.000,0.8917,1,XBAN,EUR,2506042,0.8917,37000,0.8917
COMMERZBANK AG,Common stock,07:00,DE000CBK1001,11.04,2018-03-26 00:00:00.000,11.058,17,CBK,EUR,2504665,11.09,23698,11.09
BEATE UHSE AG,Common stock,07:03,DE0007551400,0.02,2018-03-26 00:00:00.000,0.0205,4,USE,EUR,2505107,0.0205,245598,0.02
E.ON SE NA O.N.,Common stock,07:02,DE000ENAG999,8.978,2018-03-26 00:00:00.000,8.98,37,EOAN,EUR,2504666,8.996,20376,8.995
COMMERZBANK ETC UNL.,ETC,07:31,DE000ETC0290,2.379,2018-03-26 00:00:00.000,2.379,1,X0D1,EUR,2506313,2.379,54500,2.379
DK EO STOXX SEL.DIVID.30,ETF,07:11,DE000ETFL078,19.924,2018-03-26 00:00:00.000,19.924,2,EL4G,EUR,2506378,19.924,40599,19.924
"STEINHOFF INT.HLDG.EO-,50",Common stock,07:02,NL0011375019,0.2535,2018-03-26 00:00:00.000,0.2555,27,SNH,EUR,2506267,0.2596,166227,0.254
ISHSIV-FALL.A.H.Y.C.BDDLD,ETF,07:17,IE00BYM31M36,4.4031,2018-03-26 00:00:00.000,4.4031,1,QDVQ,EUR,2505524,4.4031,195000,4.4031


# Remove Data
When you are done - cleaning the directory will be done by running the following:

In [8]:
# unmark the comment
# !rm -rf $HOME/examples/*


In order to release compute and memory resources taken by spark we recommend running the following command 

In [ ]:
spark.stop()